In [1]:

%load_ext bigdata

In [54]:
%hive_start

In [3]:

%timeout 300

In [4]:
!hdfs dfs -ls /tmp

Found 2 items
drwxrwx---   - root supergroup          0 2020-01-20 01:29 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-01-20 01:31 /tmp/hive


In [5]:
!hdfs dfs -mkdir /tmp/hivep4

In [6]:
!hdfs dfs -ls /tmp/

Found 3 items
drwxrwx---   - root supergroup          0 2020-01-20 01:29 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-01-20 01:31 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-01-20 01:31 /tmp/hivep4


In [7]:
!hdfs dfs -copyFromLocal tbl0.csv /tmp/hivep4/

In [8]:
!hdfs dfs -copyFromLocal tbl1.csv /tmp/hivep4/

In [9]:
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/hivep4

Found 2 items
-rw-r--r--   1 root supergroup        397 2020-01-20 01:31 /tmp/hivep4/tbl0.csv
-rw-r--r--   1 root supergroup        282 2020-01-20 01:31 /tmp/hivep4/tbl1.csv


In [10]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 22.976 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 7.188 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 2.964 seconds


In [12]:
%%hive
SHOW TBLPROPERTIES data;

SHOW TBLPROPERTIES data;
OK
numFiles	1
numRows	0
rawDataSize	0
totalSize	677
transient_lastDdlTime	1579481467
Time taken: 0.031 seconds, Fetched: 5 row(s)


In [56]:
%%hive
SELECT * FROM tbl0;

SELECT * FROM tbl0;
OK
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}
2	C	4	2015-05-14	["a","c"]	{"dd":40,"bb":20,"cc":10}
3	D	6	2014-12-26	["b","d"]	{"aa":10,"bb":40}
4	D	5	2016-06-25	["a","c","e","b","d"]	{"bb":40,"dd":20,"aa":10,"cc":30}
5	C	7	2016-05-23	["d","e","c"]	{"cc":10,"aa":20}
6	A	2	2018-06-14	["a","d"]	{"aa":20}
7	B	3	2014-12-22	["a","e","d"]	{"cc":20}
8	C	6	2015-08-20	["d","a","c","e"]	{"cc":10,"aa":20}
9	B	3	2017-12-08	["b","a","c","e"]	{"cc":40,"dd":10,"aa":30,"bb":20}
10	B	7	2015-07-28	["c","d","e","a","b"]	{"aa":10,"dd":40,"cc":30}
Time taken: 10.216 seconds, Fetched: 10 row(s)


In [42]:
%%hive
DROP TABLE IF EXISTS word_counts;
CREATE TABLE word_counts
AS 

select  distinct a.conver from (select explode(C5) as conver from tbl0) as a    
;


INSERT OVERWRITE DIRECTORY '/tmp/hivep4'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM word_counts;

DROP TABLE IF EXISTS word_counts;
OK
Time taken: 0.005 seconds
CREATE TABLE word_counts
AS 
 a      distinct a.conver from (select explode(C5) as conver from tbl0) as 
;
Query ID = root_20200120024240_193f2b18-52c7-49ae-a9c6-41091fd93420
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579483677500_0015, Tracking URL = http://e830e59f29ce:8088/proxy/application_1579483677500_0015/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579483677500_0015
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-20 02:42:47,500 Stage-1 map = 0%,  reduce = 0%
2020-01-20 02:42:51,818 Stage-1 ma

In [43]:
!hdfs dfs -cat /tmp/hivep4/000000_0 

a
b
c
d
e


In [44]:
!hdfs dfs -copyToLocal /tmp/hivep4/000000_0 output;

In [ ]:
########pto5

In [101]:
%%hive
DROP TABLE IF EXISTS pt5;
CREATE TABLE pt5
AS
SELECT YEAR(c4) as ano, c0
FROM tbl0 LATERAL VIEW explode(c5) adTable AS c0  

;

CREATE TABLE pt51
AS
SELECT ano, c0, count(1)
FROM pt5 
group by ano, c0

;


INSERT OVERWRITE DIRECTORY '/tmp/hivep4'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM pt51;

DROP TABLE IF EXISTS pt5;
OK
Time taken: 0.188 seconds
CREATE TABLE pt5
AS
SELECT YEAR(c4) as ano, c0
FROM tbl0 LATERAL VIEW explode(c5) adTable AS c0  
;
Query ID = root_20200120050734_112f93e5-149f-4f65-ad0f-f8ce6212d152
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1579483677500_0033, Tracking URL = http://e830e59f29ce:8088/proxy/application_1579483677500_0033/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579483677500_0033
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-01-20 05:07:40,932 Stage-1 map = 0%,  reduce = 0%
2020-01-20 05:07:47,412 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 3.02 sec
MapReduce Total cumulative CPU time: 3 seconds 20 msec
Ended Job = job_1579483677500_0033
Stage-4 is selected by condition resolver.
Stage-3 is filtered out by condition resolver.
Stage-5 is filtered out by condition resolver.
Moving data to directory hdfs:

In [102]:
!hdfs dfs -cat /tmp/hivep4/000000_0 

2014,a,1
2014,b,1
2014,d,2
2014,e,1
2015,a,3
2015,b,1
2015,c,3
2015,d,2
2015,e,2
2016,a,2
2016,b,1
2016,c,3
2016,d,3
2016,e,3
2017,a,1
2017,b,1
2017,c,1
2017,e,1
2018,a,1
2018,d,1
